In [1]:
SAMPLE_INPUT = """
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"""

In [2]:
from typing import List
from pydantic import BaseModel
from rich import print

In [3]:
class CubeSet(BaseModel):
	red: int = 0
	green: int = 0
	blue: int = 0

	def power(self):
		return self.red * self.green * self.blue
	
	@staticmethod
	def read(text: str):
		for cube_set_text in text.split('; '):
			cubes = cube_set_text.split(', ')
			cube_set = {}
			for cube in cubes:
				count, color = cube.split(' ')
				cube_set[color] = int(count)
			yield CubeSet(**cube_set)


class Game(BaseModel):
	game_id: int
	cube_sets: List[CubeSet]

	@staticmethod
	def read(text: str):
		game_id, cube_sets_text = text.split(': ')
		game_number = int(game_id.replace('Game ', ''))
		return Game(game_id=game_number, cube_sets=list(CubeSet.read(cube_sets_text)))
	
	def is_possible(self, cube_set: CubeSet):
		for game_cube_set in self.cube_sets:
			for color in ['red', 'green', 'blue']:
				if getattr(game_cube_set, color) > getattr(cube_set, color):
					return False
		return True
	
	def fewest_cubes_in_game(self):
		fewest = CubeSet()
		for game_cube_set in self.cube_sets:
			for color in ['red', 'green', 'blue']:
				if getattr(game_cube_set, color) > getattr(fewest, color):
					setattr(fewest, color, getattr(game_cube_set, color))
		return fewest
	
	def power(self):
		return self.fewest_cubes_in_game().power()
	
class GameList(BaseModel):
	games: List[Game]

	@staticmethod
	def read(text: str):
		return GameList(games=[Game.read(line) for line in text.splitlines() if line])
	
	def get_possible_games(self, cube_set: CubeSet) -> List[Game]:
		possible = [game for game in self.games if game.is_possible(cube_set)]
		print(len(possible), '/', len(self.games), '|', (len(possible)*100/len(self.games)))
		return possible
	
	def score(self, cube_set: CubeSet):
		return sum([game.game_id for game in self.get_possible_games(cube_set)])
	
	def power(self):
		return sum([game.power() for game in self.games])

In [4]:
print(GameList.read(SAMPLE_INPUT))

GameList(
    games=[
        Game(
            game_id=1,
            cube_sets=[
                CubeSet(red=4, green=0, blue=3),
                CubeSet(red=1, green=2, blue=6),
                CubeSet(red=0, green=2, blue=0)
            ]
        ),
        Game(
            game_id=2,
            cube_sets=[
                CubeSet(red=0, green=2, blue=1),
                CubeSet(red=1, green=3, blue=4),
                CubeSet(red=0, green=1, blue=1)
            ]
        ),
        Game(
            game_id=3,
            cube_sets=[
                CubeSet(red=20, green=8, blue=6),
                CubeSet(red=4, green=13, blue=5),
                CubeSet(red=1, green=5, blue=0)
            ]
        ),
        Game(
            game_id=4,
            cube_sets=[
                CubeSet(red=3, green=1, blue=6),
                CubeSet(red=6, green=3, blue=0),
                CubeSet(red=14, green=3, blue=15)
            ]
        ),
        Game(game_id=5, cube_sets=[CubeSet(red=6, green=3, blue=1), CubeSet(red=1, green=2, blue=2)])
    ]
)

In [5]:
GameList.read(SAMPLE_INPUT).score(CubeSet(red=12, green=13, blue= 14))

3 / 5 | 60.0

8

In [8]:
input_text = open('input.txt').read()

In [9]:
GameList.read(input_text).score(CubeSet(red=12, green=13, blue= 14))

49 / 100 | 49.0

2545

In [10]:
GameList.read(SAMPLE_INPUT).power()

2286

In [11]:
GameList.read(input_text).power()

78111